### Link to homework
https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/01-docker-terraform/homework.md


In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from urllib.parse import quote_plus
from time import time

In [2]:
trip_data_file_path = "/workspaces/data-engineering-zoomcamp/data/green_tripdata_2019-09.csv.gz"
green_table_name = "green_tripdata_2019_09"

In [54]:
encoded_password = quote_plus("P@ssw0rd!")
print(encoded_password)
engine = create_engine(f"postgresql://postgres:{encoded_password}@db:5432/ny_taxi")

P%40ssw0rd%21


In [55]:
engine.connect()

### Question 1. Knowing docker tags
Run the command to get information on Docker
docker --help

Now run the command to get help on the "docker build" command:
docker build --help

Do the same for "docker run".
Which tag has the following text? - Automatically remove the container when it exits

Answer:
**--rm**

### Question 2. Understanding docker first run
Run docker with the python:3.9 image in an interactive mode and the entrypoint of bash. Now check the python modules that are installed ( use pip list ).

What is version of the package wheel ?

Answer:
root@c3816d7e6e77:/# wheel version

wheel 0.45.1

### Question 3. Count records
How many taxi trips were totally made on September 18th 2019?

Tip: started and finished on 2019-09-18.

Remember that lpep_pickup_datetime and lpep_dropoff_datetime columns are in the format timestamp (date and hour+min+sec) and not in date.

15767
15612
15859
89009

In [91]:
query = f"""
SELECT COUNT(*)
FROM green_tripdata_2019_09 AS GTD
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND GTD."lpep_dropoff_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_dropoff_datetime" <= '2019-09-18 23:59:59'
"""

pd.read_sql_query(query, engine)

,count
0,15612


### Question 4. Longest trip for each day
Which was the pick up day with the longest trip distance? Use the pick up time for your calculations.

Tip: For every trip on a single day, we only care about the trip with the longest distance.

2019-09-18
2019-09-16
2019-09-26
2019-09-21


In [29]:
query = f"""
SELECT COUNT(*) FROM {green_table_name}
"""

pd.read_sql_query(query, engine)

,count
0,449063


In [108]:
query = f"""
SELECT * FROM green_tripdata_2019_09 AS GTD
ORDER BY GTD.trip_distance DESC
LIMIT 10
"""

pd.read_sql_query(query, engine)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-09-26 19:32:52,2019-09-27 01:50:27,N,4.0,265,16,1.0,341.64,1761.00,1.00,0.5,0.00,0.00,None,0.3,1762.80,2.0,1.0,0.0
1,2.0,2019-09-21 00:18:15,2019-09-21 03:38:13,N,1.0,225,265,1.0,135.53,345.00,0.50,0.5,0.00,22.74,None,0.3,369.04,2.0,1.0,0.0
2,1.0,2019-09-16 07:17:37,2019-09-16 09:31:03,N,5.0,166,265,1.0,114.30,0.00,0.00,0.0,0.00,0.00,None,0.0,0.00,2.0,2.0,0.0
3,2.0,2019-09-28 03:30:40,2019-09-28 06:06:40,N,1.0,265,265,1.0,89.64,235.50,0.50,0.5,0.00,6.12,None,0.3,242.92,1.0,1.0,0.0
4,2.0,2019-09-24 01:16:42,2019-09-24 03:37:54,N,1.0,127,265,1.0,82.12,217.00,0.50,0.5,0.00,0.00,None,0.3,218.30,2.0,1.0,0.0
5,NaN,2019-09-18 18:20:00,2019-09-18 19:01:00,None,NaN,127,135,NaN,70.28,42.62,2.75,0.5,0.00,6.12,None,0.0,51.99,NaN,NaN,NaN
6,2.0,2019-09-10 06:49:56,2019-09-10 08:10:06,N,5.0,265,265,1.0,69.67,0.01,0.00,0.0,0.00,0.00,None,0.0,0.01,1.0,2.0,0.0
7,2.0,2019-09-27 17:51:39,2019-09-27 19:32:35,N,5.0,265,265,1.0,68.41,138.00,0.00,0.0,30.10,12.50,None,0.0,180.60,1.0,2.0,0.0
8,2.0,2019-09-28 01:21:42,2019-09-28 03:13:01,N,1.0,260,265,1.0,67.20,173.50,0.50,0.5,38.28,16.62,None,0.3,229.70,1.0,1.0,0.0
9,2.0,2019-09-02 03:56:18,2019-09-02 05:15:51,N,5.0,80,265,4.0,61.69,200.00,0.00,0.0,0.00,22.74,None,0.0,222.74,2.0,2.0,0.0


Answer: the longest trip was on 2019-09-26

### Question 5. Three biggest pick up Boroughs (WRONG SOLUTION)
Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

"Brooklyn" "Manhattan" "Queens";
"Bronx" "Brooklyn" "Manhattan";
"Bronx" "Manhattan" "Queens";
"Brooklyn" "Queens" "Staten Island";

In [ ]:
query = f"""
SELECT SUM(total_amount) AS total_amount, 'Brooklyn/Manhattan/Queens' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Brooklyn' OR T.Borough = 'Manhattan' OR T.Borough = 'Queens')

UNION

SELECT SUM(total_amount) AS total_amount, 'Bronx/Brooklyn/Manhattan' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Bronx' OR T.Borough = 'Brooklyn' OR T.Borough = 'Manhattan')

UNION

SELECT SUM(total_amount) AS total_amount, 'Bronx/Manhattan/Queens' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Bronx' OR T.Borough = 'Manhattan' OR T.Borough = 'Queens')

UNION

SELECT SUM(total_amount) AS total_amount, 'Brooklyn/Queens/Staten Island' as group_name
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS T ON GTD."PULocationID" = T."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (T.Borough = 'Brooklyn' OR T.Borough = 'Queens' OR T.Borough = 'Staten Island')
"""
# --

pd.read_sql(query, engine)

,total_amount,group_name
0,221434.63,Bronx/Brooklyn/Manhattan
1,268136.34,Brooklyn/Manhattan/Queens
2,176207.63,Brooklyn/Queens/Staten Island
3,204633.19,Bronx/Manhattan/Queens


### Question 5. Three biggest pick up Boroughs (CORRECT SOLUTION)
Consider lpep_pickup_datetime in '2019-09-18' and ignoring Borough has Unknown

Which were the 3 pick up Boroughs that had a sum of total_amount superior to 50000?

"Brooklyn" "Manhattan" "Queens";
"Bronx" "Brooklyn" "Manhattan";
"Bronx" "Manhattan" "Queens";
"Brooklyn" "Queens" "Staten Island";

In [116]:
query = f"""
SELECT TZ."borough", SUM(total_amount) AS total_amount
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS TZ ON GTD."PULocationID" = TZ."LocationID"
WHERE GTD."lpep_pickup_datetime" >= '2019-09-18 00:00:00'
  AND GTD."lpep_pickup_datetime" <= '2019-09-18 23:59:59'
  AND (TZ.borough = 'Brooklyn'
    OR TZ.borough = 'Manhattan'
    OR TZ.borough = 'Queens'
    OR TZ.borough = 'Bronx'
    OR TZ.borough = 'Staten Island')
GROUP BY TZ."borough"
ORDER BY SUM(total_amount) DESC
"""
# --

pd.read_sql(query, engine)

,borough,total_amount
0,Brooklyn,96333.24
1,Manhattan,92271.30
2,Queens,79531.80
3,Bronx,32830.09
4,Staten Island,342.59


Answer to question 5 is "Brooklyn" "Manhattan" "Queens"

### Question 6. Largest tip
For the passengers picked up in September 2019 in the zone name Astoria which was the drop off zone that had the largest tip? We want the name of the zone, not the id.

Note: it's not a typo, it's tip , not trip

Central Park
Jamaica
JFK Airport
Long Island City/Queens Plaza

In [65]:
query = f"""
SELECT *
FROM taxi_zones AS TZ
WHERE TZ."zone" = 'Astoria'
LIMIT 10
"""

pd.read_sql(query, engine)

,zone,LocationID,borough
0,Astoria,7,Queens


In [84]:
query = f"""
SELECT GTD."DOLocationID"
FROM green_tripdata_2019_09 AS GTD
INNER JOIN taxi_zones AS TZ ON GTD."PULocationID" = TZ."LocationID"
WHERE TZ."zone" = 'Astoria'
ORDER BY GTD."tip_amount" DESC
LIMIT 1
"""

pd.read_sql(query, engine)

,DOLocationID
0,132


In [88]:
query = f"""
WITH top_tip_location_id (location_id) AS (
  SELECT GTD."DOLocationID"
  FROM green_tripdata_2019_09 AS GTD
  INNER JOIN taxi_zones AS TZ ON GTD."PULocationID" = TZ."LocationID"
  WHERE TZ."zone" = 'Astoria'
  ORDER BY GTD."tip_amount" DESC
  LIMIT 1
)
SELECT TZ.zone
FROM taxi_zones AS TZ, top_tip_location_id AS TTLI
WHERE TZ."LocationID" = TTLI."location_id"
"""

pd.read_sql(query, engine)

,zone
0,JFK Airport
